In [ ]:
# Price & Trades Analysis
# Interactive visualization of OHLC data with trade entries and exits

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler
from plotly_resampler.aggregation import MinMaxLTTB
import os
import sys

# Add research directory to path
sys.path.append('/home/stan/src/download-polygon-aggregates/research')
from data_downloader import get_filename

In [ ]:
# Load OHLC bars and trades data
# Configuration
ticker = 'AAPL'
interval = '5s'
year = '2024'

# Load OHLC data
pwd = '/home/stan/src/download-polygon-aggregates/research/dash'
filename = get_filename(ticker, interval, year)
bars = pd.read_parquet(f'{pwd}/../{filename}')
bars['timestamp'] = pd.to_datetime(bars['timestamp'], unit='s')

# Load trades data
trades = pd.read_csv(f"{pwd}/trades.csv", parse_dates=["entry_ts", "exit_ts"]).sort_values("entry_ts")

print(f"Loaded {len(bars):,} bars and {len(trades):,} trades")
print(f"Data period: {bars['timestamp'].min()} to {bars['timestamp'].max()}")
print(f"Trades period: {trades['entry_ts'].min()} to {trades['exit_ts'].max()}")

In [ ]:
# Helper functions for filtering trades and creating charts

def filter_trades(trades_df, params_hash=None, stop_loss=None, side="ALL"):
    """Filter trades by parameters"""
    df = trades_df.copy()
    
    if params_hash is not None:
        df = df[df["params_hash"].astype(str) == str(params_hash)]
    
    if stop_loss is not None:
        df = df[df["stop_loss"] == stop_loss]
    
    if side == "LONG":
        df = df[df["side"].str.lower() == "long"]
    elif side == "SHORT":
        df = df[df["side"].str.lower() == "short"]
    
    return df

def create_price_chart(bars_df, trades_df):
    """Create price chart with trades overlay using FigureWidgetResampler for scroll zoom + range tracking"""
    from plotly_resampler import FigureWidgetResampler
    
    timestamps_np = bars_df["timestamp"].values
    
    # Create FigureWidgetResampler instance
    fig = FigureWidgetResampler(go.FigureWidget())
    
    # Add price data
    fig.add_trace(
        go.Scattergl(mode="lines", name="Close", line=dict(width=1)),
        hf_x=timestamps_np, hf_y=bars_df["close"]
    )
    fig.add_trace(
        go.Scattergl(mode="lines", name="High", line=dict(width=1, color="gray")),
        hf_x=timestamps_np, hf_y=bars_df["high"]
    )
    fig.add_trace(
        go.Scattergl(mode="lines", name="Low", line=dict(width=1, color="gray")),
        hf_x=timestamps_np, hf_y=bars_df["low"]
    )
    
    # Add trade markers
    if len(trades_df) > 0:
        fig.add_trace(
            go.Scattergl(mode="markers", name="Entry", 
                        marker=dict(symbol="triangle-up", size=8, color="blue")),
            hf_x=trades_df["entry_ts"].values, hf_y=trades_df["entry_price"]
        )
        
        exit_colors = np.where(trades_df["pnl"] >= 0, "green", "red")
        fig.add_trace(
            go.Scattergl(mode="markers", name="Exit",
                        marker=dict(symbol="x", size=8, color=exit_colors),
                        text=[f"PnL: {pnl:.2f}" for pnl in trades_df["pnl"]],
                        hovertemplate="%{text}<extra></extra>"),
            hf_x=trades_df["exit_ts"].values, hf_y=trades_df["exit_price"]
        )
    
    fig.update_layout(
        height=600,
        margin=dict(l=50, r=50, t=20, b=20),
        xaxis_rangeslider_visible=False,
        showlegend=False,
        dragmode='pan',
        hovermode='x unified'
    )
    
    # Enable scroll zoom
    fig._config = {'scrollZoom': True}
    
    return fig

def get_zoomed_trades(trades, fig):
    """Get trades filtered by current zoom range on the chart"""
    
    if fig is None:
        print("No chart available")
        return trades
    
    # Get current visible range from the chart
    current_range = fig.layout.xaxis.range
    
    if not current_range:
        return trades
    
    if len(trades) == 0:
        return trades
    
    # Convert range to datetime objects
    start_time = pd.to_datetime(current_range[0])
    end_time = pd.to_datetime(current_range[1])
    
    # Filter trades by entry time within visible range
    zoomed_trades = trades[
        (trades["entry_ts"] >= start_time) & 
        (trades["entry_ts"] <= end_time)
    ]
    return zoomed_trades

def show_trades_table(trades, fig):
    """Show trades table for current zoom range"""
    zoomed_trades = get_zoomed_trades(trades, fig)
    
    if len(zoomed_trades) == 0:
        print("No trades in current zoom range")
        return
    
    trades_display = zoomed_trades[[
        'trade_id', 'side', 'entry_ts', 'entry_price', 
        'exit_ts', 'exit_price', 'pnl', 'pnl_pct', 
        'stop_loss', 'duration_s'
    ]].round(5)
    display(trades_display)

    print(f"Trades in zoom range: {len(zoomed_trades)}")
    print(f"Total PnL: {zoomed_trades['pnl'].sum():.2f}")
    print(f"Win rate: {(zoomed_trades['pnl'] > 0).mean():.1%}")
    print(f"Avg PnL: {zoomed_trades['pnl'].mean():.2f}")

# Get available filter options
params_options = sorted(trades["params_hash"].astype(str).unique().tolist())
stoploss_options = sorted(trades["stop_loss"].unique().tolist())
side_options = ["ALL", "LONG", "SHORT"]

print(f"Available params_hash: {params_options[:5]}... ({len(params_options)} total)")
print(f"Available stop_loss: {stoploss_options}")
print(f"Available sides: {side_options}")

In [ ]:
PARAMS_HASH = params_options[0]
STOP_LOSS = stoploss_options[0]
SIDE = "ALL"

custom_trades = []
def apply_filters():
    global custom_trades
    custom_trades = filter_trades(trades, 
                                params_hash=PARAMS_HASH, 
                                stop_loss=STOP_LOSS, 
                                side=SIDE)


In [ ]:
PARAMS_HASH = params_options[0]
STOP_LOSS = stoploss_options[0]
SIDE = "ALL" # "ALL", "LONG", or "SHORT"

apply_filters()
fg = create_price_chart(bars, custom_trades)
fg  # Display the chart

In [ ]:
show_trades_table(custom_trades, fg)  # Shows only trades in zoomed range

In [ ]:
# Quick statistics summary
if not custom_trades.empty:
    stats = {
        'Total Trades': len(custom_trades),
        'Total PnL': custom_trades['pnl'].sum(),
        'Win Rate': (custom_trades['pnl'] > 0).mean(),
        'Average PnL': custom_trades['pnl'].mean(),
        'Best Trade': custom_trades['pnl'].max(),
        'Worst Trade': custom_trades['pnl'].min(),
        'Avg Duration (s)': custom_trades['duration_s'].mean(),
        'Long Trades': (custom_trades['side'].str.lower() == 'long').sum(),
        'Short Trades': (custom_trades['side'].str.lower() == 'short').sum(),
    }
    
    stats_df = pd.DataFrame(list(stats.items()), columns=['Metric', 'Value'])
    display(stats_df)
else:
    print("No trades found for selected filters")